In [11]:
#Now lets start with creating an end-to-end chatbot using Python.

In [12]:
import os
import nltk
import ssl
import streamlit as st
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath("nltk_data"))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vhema\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
#Now let’s define some intents of the chatbot. 

In [14]:
intents = [
    {
      "tag": "logo",
      "patterns": ["What is your logo?", "Can you tell me about your logo?", "Describe your logo", "What does your logo represent?"],
      "responses": ["Our logo reflects the modern, innovative approach we take to interior design."]
    },
    {
      "tag": "about_us",
      "patterns": ["Tell me about your company", "Who are you?", "What is your company about?", "Tell me more about your company","What is this website about?"],
      "responses": ["We are a leading interior design firm dedicated to transforming spaces into beautiful, functional designs.","We have an interior design website which is passionate about creating beautiful spaces."]
    },
    {
      "tag": "residential_design",
      "patterns": ["Tell me about residential design", "What is your approach to residential design?", "Do you do home interiors?", "Show me examples of residential designs"],
      "responses": ["Our residential designs are crafted to make your home feel comfortable, personalized, and stylish."]
    },
    {
      "tag": "office_design",
      "patterns": ["Tell me about office design", "Can you help with office interiors?", "What is your approach to office design?", "Show me examples of office designs"],
      "responses": ["We create office spaces that are functional, modern, and tailored to your business needs."]
    },
    {
      "tag": "conceptual_design",
      "patterns": ["What is conceptual design?", "Tell me about conceptual design", "How do you approach conceptual design?", "Give me an example of conceptual design"],
      "responses": ["Our conceptual designs start with a creative idea that turns into a full-fledged, detailed design."]
    },
    {
      "tag": "reviews",
      "patterns": ["What do clients say about you?", "Do you have reviews?", "Can I read some reviews?", "Tell me what people say about your service"],
      "responses": ["We have received excellent feedback from our clients, praising our creativity and professionalism."]
    },
    {
      "tag": "design_tips",
      "patterns": ["Do you have design tips?", "Give me some design tips", "What are some interior design tips?", "Tell me some decorating tips"],
      "responses": ["1. Use neutral colors to create a calm atmosphere. 2. Invest in quality furniture. 3. Focus on lighting to enhance the space."]
    },
    {
      "tag": "gallery",
      "patterns": ["Can I see your gallery?", "Where can I see your designs?", "Show me your design gallery", "Do you have an image gallery?"],
      "responses": ["Check out our gallery page for a collection of our past design projects."]
    },
    {
      "tag": "login",
      "patterns": ["How do I login?", "Can I login to your site?", "Where can I login?", "How do I access my account?"],
      "responses": ["You can login by clicking the 'Login' button at the top right corner of the website."]
    },
    {
      "tag": "offer",
      "patterns": ["Do you have any offers?", "What offers do you have?", "Are there any discounts?", "Do you provide special offers?"],
      "responses": ["We have seasonal offers and promotions! Check our 'Offers' section for details."]
    },
    {
      "tag": "design_process",
      "patterns": ["What is your design process?", "How do you approach a design project?", "Tell me about your design process", "How do you begin a new project?"],
      "responses": ["Our design process includes understanding your needs, creating a concept, finalizing designs, and implementing the plan."]
    },
    {
      "tag": "time",
      "patterns": ["How long does the design process take?", "What is the timeline for my design?", "How much time do you need for a project?", "How long will it take to complete my design?"],
      "responses": ["The timeline depends on the scope of the project, but most designs take between 2-4 weeks."]
    },
    {
      "tag": "examples",
      "patterns": ["Can you show me some examples?", "Do you have any examples of your work?", "Show me some design examples", "Give me an example of your work"],
      "responses": ["Sure, we have a variety of examples from residential, office, and conceptual designs. Check our portfolio."]
    },
    {
      "tag": "faqs",
      "patterns": ["Where can I find FAQs?", "Do you have a FAQ page?", "What are the most asked questions?", "Tell me about your FAQ section"],
      "responses": ["You can find answers to common questions on our 'FAQ' page."]
    },
    {
      "tag": "consultation",
      "patterns": ["Do you offer consultations?", "Can I schedule a consultation?", "How can I book a consultation?", "Tell me about your consultation services"],
      "responses": ["Yes, we offer consultations. You can book a consultation by filling out the form on our website."]
    },
    {
      "tag": "bedroom_decor",
      "patterns": ["Give me bedroom decor ideas", "Tell me how to decorate my bedroom", "What are some bedroom design tips?", "How can I decorate my bedroom?"],
      "responses": ["Consider using soft, calming colors and functional furniture. Add textiles like rugs and cushions to make it cozy."]
    },
    {
      "tag": "traditional_designs",
      "patterns": ["Tell me about traditional designs", "What are traditional design styles?", "Give me ideas for traditional decor", "What is traditional interior design?"],
      "responses": ["Traditional designs focus on timeless styles with rich colors, classic furniture, and ornate details."]
    },
    {
      "tag": "color_psychology",
      "patterns": ["Tell me about color psychology", "What do colors mean in interior design?", "How do colors affect mood?", "Can you explain color psychology?"],
      "responses": ["Colors can evoke different emotions. For example, blue promotes calmness, while red encourages energy."]
    },
    {
      "tag": "small_space_ideas",
      "patterns": ["What are some small space design ideas?", "How do I make the most of a small space?", "Can you help with small apartment design?", "Give me tips for small space interiors"],
      "responses": ["Use multi-functional furniture, opt for light colors, and keep the space clutter-free to maximize space."]
    },
    {
      "tag": "accessories",
      "patterns": ["What accessories should I add?", "Can you suggest some home decor accessories?", "Tell me about interior design accessories", "What are some accessories for home design?"],
      "responses": ["Consider adding throw pillows, rugs, wall art, and stylish lighting to enhance your space."]
    },
    {
      "tag": "design_hacks",
      "patterns": ["Do you have any design hacks?", "Give me some interior design tricks", "What are some design shortcuts?", "Tell me some design hacks for my home"],
      "responses": ["Use mirrors to make small spaces appear larger. Light-colored walls can brighten a room and make it feel more spacious."]
    },
    {
      "tag": "greeting",
      "patterns": [
        "Hi",
        "Hello",
        "Hey",
        "Good morning",
        "Good evening",
        "How are you?"
      ],
      "responses": [
        "Hello! How can I assist you with your interior design needs?",
        "Hi there! Looking for some interior design inspiration?",
        "Hey! Tell me how I can help you today."
      ]
    },
    {
      "tag": "services",
      "patterns": [
        "What services do you offer?",
        "Tell me about your services",
        "What can you do for me?",
        "Do you provide consultations?",
        "What interior design services do you have?"
      ],
      "responses": [
        "We offer design consultations, furniture selection, space planning, and full project management.",
        "Our services include home decor consultations, custom furniture design, and renovation planning.",
        "We specialize in transforming spaces to suit your style and needs."
      ]
    },
    {
      "tag": "pricing",
      "patterns": [
        "How much do you charge?",
        "Tell me about your prices",
        "What are your rates?",
        "Is it expensive?",
        "Do you have an hourly rate?"
      ],
      "responses": [
        "Our pricing varies based on the project scope. Please contact us for a detailed quote.",
        "We offer competitive rates tailored to your project's requirements.",
        "Reach out to us for a free consultation and pricing estimate."
      ]
    },
    {
      "tag": "portfolio",
      "patterns": [
        "Can I see your portfolio?",
        "Show me some examples of your work",
        "Do you have a portfolio?",
        "I want to see your past projects"
      ],
      "responses": [
        "Certainly! Visit our portfolio section on the website to explore our projects.",
        "You can check out our work in the portfolio section of our website.",
        "We have an extensive portfolio showcasing our designs. Visit our portfolio page for details."
      ]
    },
    {
      "tag": "appointment",
      "patterns": [
        "How do I book a consultation?",
        "Can I schedule an appointment?",
        "I want to meet with you",
        "How can I book a consultation?"
      ],
      "responses": [
        "You can book a consultation through our contact form on the website.",
        "To schedule a meeting, please call us directly or use the appointment feature on our site.",
        "Booking a consultation is easy! Visit our contact page to get started."
      ]
    },
    {
      "tag": "contact_info",
      "patterns": [
        "What is your contact information?",
        "How do I get in touch with you?",
        "What is your email address?",
        "Can I call you?"
      ],
      "responses": [
        "You can reach us via email at contact@interiordesign.com or call us at (123) 456-7890.",
        "Feel free to contact us through our website's contact form or at (123) 456-7890.",
        "Our contact details are on the website. Drop us a message anytime!"
      ]
    },
    {
      "tag": "room_design",
      "patterns": [
        "Can you design a bedroom?",
        "Do you offer kitchen design services?",
        "What about living room decor?",
        "Can you help with bathroom renovations?"
      ],
      "responses": [
        "Yes, we design all types of spaces, including bedrooms, kitchens, living rooms, and bathrooms.",
        "Our expertise covers every room in your home. Let's create something beautiful together.",
        "From kitchens to bedrooms, we provide comprehensive interior design services."
      ]
    },
    {
      "tag": "style_preferences",
      "patterns": [
        "Do you do modern designs?",
        "I want a traditional look. Can you help?",
        "What styles do you work with?",
        "Can you customize designs to my style?"
      ],
      "responses": [
        "Yes! We specialize in various styles, including modern, traditional, contemporary, and eclectic.",
        "Tell us your style preferences, and we’ll create something unique for you.",
        "We work with all styles and can tailor the design to match your taste."
      ]
    },
    {
      "tag": "goodbye",
      "patterns": [
        "Bye",
        "Goodbye",
        "See you later",
        "I’m done for now"
      ],
      "responses": [
        "Goodbye! Feel free to reach out if you have more questions.",
        "Take care! I hope to help you again soon.",
        "See you later! Let us know if you need more assistance."
      ]
    },
    {
      "tag": "timeline",
      "patterns": [
        "How long does a project take?",
        "What’s the average timeline?",
        "Can you complete a project quickly?",
        "How much time do you need for designing?"
      ],
      "responses": [
        "The timeline depends on the project's scope. Contact us for a tailored estimate.",
        "Small projects can take a few weeks, while larger renovations may take longer.",
        "We aim to complete projects efficiently without compromising quality."
      ]
    },
    {
        "tag": "thanks",
        "patterns": [
            "Thank you",
            "Thanks",
            "Thanks a lot",
            "I appreciate it"
        ],
        "responses": [
            "You're welcome",
            "No problem",
            "Glad I could help"
        ]
    },
    {
        "tag": "about",
        "patterns": [
            "What can you do",
            "Who are you",
            "What are you",
            "What is your purpose"
        ],
        "responses": [
            "I am a chatbot",
            "My purpose is to assist you",
            "I can answer questions and provide assistance"
        ]
    },
    {
        "tag": "help",
        "patterns": [
            "Help",
            "I need help",
            "Can you help me",
            "What should I do"
        ],
        "responses": [
            "Sure, what do you need help with?",
            "I'm here to help. What's the problem?",
            "How can I assist you?"
        ]
    },
    {
        "tag": "budget",
        "patterns": [
            "How can I make a budget",
            "What's a good budgeting strategy",
            "How do I create a budget"
        ],
        "responses": [
            "To make a budget, start by tracking your income and expenses. Then, allocate your income towards essential expenses like rent, food, and bills. Next, allocate some of your income towards savings and debt repayment. Finally, allocate the remainder of your income towards discretionary expenses like entertainment and hobbies.",
            "A good budgeting strategy is to use the 50/30/20 rule. This means allocating 50% of your income towards essential expenses, 30% towards discretionary expenses, and 20% towards savings and debt repayment.",
            "To create a budget, start by setting financial goals for yourself. Then, track your income and expenses for a few months to get a sense of where your money is going. Next, create a budget by allocating your income towards essential expenses, savings and debt repayment, and discretionary expenses."
        ]
    },
    {
        "tag": "name",
        "patterns": [
            "What's your name",
            "Do you have a name",
            "What should I call you"
        ],
        "responses": [
            "You can call me Chatbot.",
            "My name is Chatbot."
        ]
    },
    {
        "tag": "favorite_color",
        "patterns": [
            "What's your favorite color",
            "Do you like any color"
        ],
        "responses": [
            "I'm a chatbot, so I don't have a favorite color.",
            "As an AI, I don't have preferences like humans do."
        ]
    },
    {
        "tag": "hobby",
        "patterns": [
            "What do you do for fun",
            "Do you have any hobbies"
        ],
        "responses": [
            "I don't have hobbies as I'm here to assist you.",
            "I'm always ready to chat with you, so that's my favorite thing to do!"
        ]
    },
    {
        "tag": "joke",
        "patterns": [
            "Tell me a joke",
            "Do you know any jokes",
            "Make me laugh"
        ],
        "responses": [
            "Why don’t scientists trust atoms? Because they make up everything!",
            "Why don't skeletons fight each other? They don't have the guts!",
            "Did you hear about the mathematician who’s afraid of negative numbers? He’ll stop at nothing to avoid them!"
        ]
    },
    {
        "tag": "food",
        "patterns": [
            "What's your favorite food",
            "Do you like to eat",
            "Tell me about food"
        ],
        "responses": [
            "As a chatbot, I don't eat, but I'm here to help with any questions you have about food!",
            "I may not have taste buds, but I can assist you in finding recipes or restaurants!"
        ]
    },
    {
        "tag": "movies",
        "patterns": [
            "What's your favorite movie",
            "Recommend me a movie",
            "Tell me about movies"
        ],
        "responses": [
            "As an AI, I don't watch movies, but I can suggest some popular ones like The Shawshank Redemption, Inception, or The Godfather.",
            "I may not have personal preferences, but I can help you find movies based on your taste!"
        ]
    },
    {
        "tag": "technology",
        "patterns": [
            "What's the latest tech news",
            "Tell me about technology",
            "What's new in tech"
        ],
        "responses": [
            "As an AI language model, I don't have real-time data, but you can check technology news websites for the latest updates.",
            "Technology is constantly evolving. Stay updated with tech blogs and news sites to know about the latest advancements!"
        ]
    },
    {
        "tag": "compliment",
        "patterns": [
            "You're awesome",
            "You're great",
            "I like you"
        ],
        "responses": [
            "Thank you! I'm just a program, but I'm here to assist and make your experience better.",
            "I'm glad you think so! My purpose is to help and provide useful information."
        ]
    },
    {
        "tag": "meaning_of_life",
        "patterns": [
            "What is the meaning of life",
            "Why are we here",
            "What's the purpose of life"
        ],
        "responses": [
            "The meaning of life is a philosophical question. Different people and cultures have different beliefs about it.",
            "The purpose of life is subjective and can vary from person to person. Some find purpose in relationships, careers, or helping others."
        ]
    },
    {
        "tag": "travel",
        "patterns": [
            "Where is a good place to travel",
            "Recommend me a travel destination",
            "Tell me about your favorite travel spot"
        ],
        "responses": [
            "I'm just a chatbot, so I don't travel, but there are many beautiful destinations around the world to explore. It depends on your preferences, such as beaches, mountains, historical sites, or bustling cities.",
            "Traveling allows you to experience different cultures, try new cuisines, and create lasting memories."
        ]
    },
    {
        "tag": "books",
        "patterns": [
            "What's your favorite book",
            "Recommend me a book to read",
            "Tell me about books"
        ],
        "responses": [
            "As an AI, I don't have personal preferences, but there are countless amazing books in various genres. Some popular ones include Harry Potter, To Kill a Mockingbird, and 1984.",
            "Reading books can broaden your knowledge, enhance creativity, and provide a great way to relax."
        ]
    },
    {
        "tag": "education",
        "patterns": [
            "Tell me about education",
            "What's the importance of education",
            "How to study effectively"
        ],
        "responses": [
            "Education is the process of acquiring knowledge, skills, values, and attitudes. It plays a crucial role in personal and societal development.",
            "Studying effectively involves setting clear goals, creating a conducive study environment, staying organized, taking regular breaks, and seeking help when needed."
        ]
    },
    {
        "tag": "coding",
        "patterns": [
            "Tell me about coding",
            "How to start coding",
            "What's the best programming language"
        ],
        "responses": [
            "Coding is the process of writing instructions for computers to execute tasks. It powers software, websites, and apps.",
            "If you want to start coding, choose a programming language like Python, Java, or JavaScript, and explore online tutorials and courses."
        ]
    },
    {
        "tag": "art",
        "patterns": [
            "Tell me about art",
            "What's your favorite artwork",
            "How to appreciate art"
        ],
        "responses": [
            "Art comes in various forms, such as paintings, sculptures, music, literature, and more. It's a way to express emotions and ideas.",
            "Appreciating art involves being open-minded, observing details, understanding the context, and exploring different art movements."
        ]
    },
    {
        "tag": "technology_help",
        "patterns": [
            "How can you assist with technology",
            "Tell me about tech support",
            "Can you help with computer issues"
        ],
        "responses": [
            "As an AI chatbot, I can provide information and basic troubleshooting for common technology problems.",
            "For complex technical issues, it's best to consult a qualified IT professional."
        ]
    },
    {
        "tag": "history",
        "patterns": [
            "Tell me about history",
            "What's your favorite historical period",
            "How can I learn about historical events"
        ],
        "responses": [
            "History is the study of past events, societies, and civilizations. It provides insights into our roots and informs our future.",
            "Learning about history can involve reading books, visiting museums, watching documentaries, and attending history lectures."
        ]
    },
    {
        "tag": "music",
        "patterns": [
            "What's your favorite music",
            "Recommend me a song",
            "Tell me about music genres"
        ],
        "responses": [
            "As an AI, I don't have personal preferences, but there are various music genres like pop, rock, classical, hip-hop, and more.",
            "Music can evoke emotions, improve mood, and be a source of inspiration."
        ]
    },
    {
        "tag": "mindfulness",
        "patterns": [
            "What is mindfulness",
            "Tell me about mindfulness techniques",
            "How to practice mindfulness"
        ],
        "responses": [
            "Mindfulness is the practice of being fully present in the moment, observing thoughts and feelings without judgment.",
            "Mindfulness techniques include meditation, deep breathing, body scanning, and mindful eating."
        ]
    },
    {
        "tag": "positivity",
        "patterns": [
            "Spread some positivity",
            "Tell me a positive quote",
            "How to stay positive"
        ],
        "responses": [
            "Every day is a new opportunity to make a positive impact. You are capable of great things!",
            "Remember, you are strong, resilient, and capable of overcoming challenges."
        ]
    },
    {
        "tag": "nature",
        "patterns": [
            "Tell me about nature",
            "What's your favorite natural wonder",
            "How to protect the environment"
        ],
        "responses": [
            "Nature encompasses the beauty and diversity of the world, including landscapes, wildlife, and ecosystems.",
            "Protecting the environment involves reducing waste, conserving resources, and supporting sustainable practices."
        ]
    },
    {
        "tag": "productivity",
        "patterns": [
            "How to be more productive",
            "Tell me about time management",
            "Productivity tips"
        ],
        "responses": [
            "To be more productive, prioritize tasks, set goals, eliminate distractions, and take regular breaks.",
            "Effective time management can lead to increased efficiency and reduced stress."
        ]
    },
    {
        "tag": "travel_tips",
        "patterns": [
            "Tell me about travel tips",
            "How to pack for a trip",
            "What to consider while traveling"
        ],
        "responses": [
            "Travel tips include packing light, carrying essential documents, researching your destination, and respecting local customs.",
            "While traveling, be open to new experiences, try local cuisines, and be mindful of your surroundings."
        ]
    },
    {
        "tag": "languages",
        "patterns": [
            "Tell me about languages",
            "How to learn a new language",
            "What's the importance of multilingualism"
        ],
        "responses": [
            "Languages are a crucial aspect of human communication and culture, with thousands of languages spoken worldwide.",
            "Learning a new language can expand your horizons, enhance cognitive abilities, and facilitate cross-cultural interactions."
        ]
    },
    {
        "tag": "inspiration",
        "patterns": [
            "I need some inspiration",
            "Tell me an inspiring story",
            "How to stay motivated"
        ],
        "responses": [
            "You are capable of achieving remarkable things. Believe in yourself and never give up!",
            "The journey to success may have challenges, but it's the persistence and determination that lead to extraordinary accomplishments."
        ]
    },
    {
        "tag": "finance_tips",
        "patterns": [
            "Tell me about finance tips",
            "How to save money",
            "Investment advice"
        ],
        "responses": [
            "Finance tips include budgeting, saving a portion of your income, avoiding unnecessary expenses, and investing wisely for the future.",
            "Investing in stocks, mutual funds, or real estate can help grow your wealth over time."
        ]
    },
    {
        "tag": "artificial_intelligence",
        "patterns": [
            "Tell me about artificial intelligence",
            "How does AI work",
            "AI applications"
        ],
        "responses": [
            "Artificial intelligence is the simulation of human intelligence in machines, enabling them to learn and perform tasks.",
            "AI has diverse applications, including voice assistants, self-driving cars, recommendation systems, and medical diagnostics."
        ]
    },
    {
        "tag": "motivation",
        "patterns": [
            "I need some motivation",
            "Tell me an encouraging quote",
            "How to overcome obstacles"
        ],
        "responses": [
            "Believe in yourself. You have the strength to overcome any obstacle and achieve your dreams!",
            "Obstacles are stepping stones to success. Embrace challenges as opportunities for growth and learning."
        ]
    },
    {
        "tag": "future",
        "patterns": [
            "Tell me about the future",
            "What will the future look like",
            "Future technology"
        ],
        "responses": [
            "The future is uncertain, but advancements in technology, medicine, and space exploration hold great potential.",
            "The future of technology may include AI advancements, renewable energy solutions, and even the possibility of space colonization."
        ]
    },
    {
        "tag": "self_improvement",
        "patterns": [
            "How to improve myself",
            "Tell me about personal growth",
            "Tips for self-development"
        ],
        "responses": [
            "Self-improvement involves setting goals, embracing continuous learning, staying positive, and practicing self-care.",
            "Embrace challenges as opportunities for growth, and focus on becoming the best version of yourself."
        ]
    },
    {
        "tag": "robotics",
        "patterns": [
            "Tell me about robotics",
            "How do robots work",
            "Future of robotics"
        ],
        "responses": [
            "Robotics is the study and development of robots, which are machines capable of performing tasks autonomously or semi-autonomously.",
            "The future of robotics may include advancements in AI, human-robot collaboration, and robotics in industries like healthcare and manufacturing."
        ]
    },
    {
        "tag": "philosophy",
        "patterns": [
            "Tell me about philosophy",
            "What's the purpose of life",
            "Philosophical questions"
        ],
        "responses": [
            "Philosophy is the study of fundamental questions about existence, knowledge, values, reason, mind, and language.",
            "Philosophical questions explore the nature of reality, ethics, free will, consciousness, and the meaning of life."
        ]
    },
    {
        "tag": "coding_languages",
        "patterns": [
            "Tell me about coding languages",
            "Which programming language to learn",
            "Popular coding languages"
        ],
        "responses": [
            "Coding languages include Python, Java, JavaScript, C++, and more. The choice depends on your interests and the application you want to develop.",
            "Python is often recommended for beginners due to its readability and versatility."
        ]
    },
    {
        "tag": "virtual_reality",
        "patterns": [
            "Tell me about virtual reality",
            "How does VR work",
            "Applications of virtual reality"
        ],
        "responses": [
            "Virtual reality is a computer-generated simulation that allows users to interact with a 3D environment.",
            "VR has applications in gaming, training, education, architecture, and therapeutic interventions."
        ]
    },
    {
        "tag": "creativity",
        "patterns": [
            "Tell me about creativity",
            "How to boost creativity",
            "Importance of creativity"
        ],
        "responses": [
            "Creativity is the ability to think outside the box, generate new ideas, and solve problems innovatively.",
            "To boost creativity, engage in activities like writing, drawing, brainstorming, or trying new experiences."
        ]
    },
    {
        "tag": "entrepreneurship",
        "patterns": [
            "Tell me about entrepreneurship",
            "How to start a business",
            "Keys to entrepreneurial success"
        ],
        "responses": [
            "Entrepreneurship involves identifying opportunities, taking risks, and creating new ventures.",
            "Success in entrepreneurship requires determination, adaptability, effective leadership, and a problem-solving mindset."
        ]
    },
    {
        "tag": "universe",
        "patterns": [
            "Tell me about the universe",
            "How big is the universe",
            "Astrology vs. astronomy"
        ],
        "responses": [
            "The universe is vast, containing billions of galaxies, each with billions of stars.",
            "Astrology is the study of celestial bodies' influence on human affairs, while astronomy is the scientific study of celestial objects and phenomena."
        ]
    },
    {
        "tag": "social_media",
        "patterns": [
            "Tell me about social media",
            "Pros and cons of social networking",
            "How to use social media responsibly"
        ],
        "responses": [
            "Social media platforms allow people to connect, share content, and stay updated on news and events.",
            "Using social media responsibly involves protecting privacy, avoiding misinformation, and fostering positive interactions."
        ]
    },
    {
        "tag": "happiness",
        "patterns": [
            "Tell me about happiness",
            "How to find happiness",
            "Keys to a happy life"
        ],
        "responses": [
            "Happiness is a subjective and individual experience, and it can be found in meaningful relationships, personal achievements, and gratitude.",
            "Focusing on positive aspects, practicing mindfulness, and helping others contribute to a happier life."
        ]
    },
    {
        "tag": "data_privacy",
        "patterns": [
            "Tell me about data privacy",
            "Why is data security important",
            "Protecting personal information online"
        ],
        "responses": [
            "Data privacy refers to safeguarding personal and sensitive information from unauthorized access and misuse.",
            "Protecting personal information online involves using strong passwords, enabling two-factor authentication, and being cautious about sharing sensitive data."
        ]
    },
    {
        "tag": "positivity_quotes",
        "patterns": [
            "Tell me a positive quote",
            "Inspirational quotes",
            "Motivational sayings"
        ],
        "responses": [
            "The future belongs to those who believe in the beauty of their dreams. - Eleanor Roosevelt",
            "In the middle of every difficulty lies opportunity. - Albert Einstein",
            "Your time is limited, don't waste it living someone else's life. - Steve Jobs"
        ]
    },
    {
        "tag": "virtual_assistant",
        "patterns": [
            "Tell me about virtual assistants",
            "How does a virtual assistant work",
            "Popular virtual assistant applications"
        ],
        "responses": [
            "Virtual assistants are AI-powered programs that perform tasks and provide information through natural language interaction.",
            "Popular virtual assistants include Siri, Alexa, Google Assistant, and Cortana."
        ]
    },
    {
        "tag": "sustainability",
        "patterns": [
            "Tell me about sustainability",
            "Why is sustainable living important",
            "How to be more eco-friendly"
        ],
        "responses": [
            "Sustainability aims to meet current needs without compromising the ability of future generations to meet their needs.",
            "Adopting eco-friendly practices, reducing waste, and supporting sustainable products contribute to a greener planet."
        ]
    },
    {
        "tag": "machine_learning",
        "patterns": [
            "Tell me about machine learning",
            "How does ML work",
            "Applications of machine learning"
        ],
        "responses": [
            "Machine learning is a subset of AI that enables systems to learn from data and improve performance over time.",
            "ML has applications in image recognition, natural language processing, recommendation systems, and predictive analytics."
        ]
    },
    {
        "tag": "home_decor",
        "patterns": [
            "Tell me about home decor ideas",
            "How to decorate a small space",
            "DIY home projects"
        ],
        "responses": [
            "Home decor ideas can involve adding plants, changing wall colors, and incorporating cozy textiles for a welcoming space.",
            "For small spaces, use multipurpose furniture, mirrors to create an illusion of space, and keep clutter to a minimum."
        ]
    },
    {
        "tag": "time_management",
        "patterns": [
            "Tell me about time management techniques",
            "How to stay productive",
            "Avoiding procrastination"
        ],
        "responses": [
            "Time management techniques include creating to-do lists, using time blocks, and setting specific goals.",
            "To stay productive, identify your most productive hours, eliminate distractions, and take regular breaks."
        ]
    },
    {
        "tag": "online_learning",
        "patterns": [
            "Tell me about online learning platforms",
            "How to stay motivated while learning online",
            "Advantages of e-learning"
        ],
        "responses": [
            "Online learning platforms like Coursera, Udemy, and edX offer a wide range of courses on various subjects.",
            "Staying motivated while learning online can be achieved by setting clear learning goals, tracking progress, and participating in online communities."
        ]
    },
    {
        "tag": "art_and_crafts",
        "patterns": [
            "Tell me about art and craft ideas",
            "How to start a DIY project",
            "Benefits of creativity"
        ],
        "responses": [
            "Art and craft ideas can include painting, drawing, knitting, or making handmade cards.",
            "Creativity enhances problem-solving skills, reduces stress, and boosts self-esteem."
        ]
    },
    {
        "tag": "budgeting_tips",
        "patterns": [
            "Tell me about frugal living",
            "How to save money on groceries",
            "Managing finances as a student"
        ],
        "responses": [
            "Frugal living involves being mindful of expenses, budgeting, and finding ways to save money without compromising on needs.",
            "To save money on groceries, make a shopping list, compare prices, and look for discounts or coupons."
        ]
    },
    {
        "tag": "personal_finances",
        "patterns": [
            "Tell me about financial planning",
            "How to start investing",
            "Retirement planning tips"
        ],
        "responses": [
            "Financial planning involves setting financial goals, creating a budget, and building an emergency fund.",
            "Before investing, research different investment options, consider risk tolerance, and diversify your portfolio."
        ]
    },
    {
        "tag": "online_security",
        "patterns": [
            "Tell me about online security tips",
            "How to create strong passwords",
            "Protecting children online"
        ],
        "responses": [
            "Online security tips include using strong passwords, enabling two-factor authentication, and avoiding public Wi-Fi for sensitive transactions.",
            "To protect children online, use parental control software, educate them about online safety, and monitor their online activities."
        ]
    },
    {
        "tag": "home_repair",
        "patterns": [
            "Tell me about common home repairs",
            "How to fix a leaky faucet",
            "Hiring professionals for home improvement"
        ],
        "responses": [
            "Common home repairs include fixing a leaky faucet, unclogging drains, and patching small holes in walls.",
            "For complex home improvements, hiring a professional can ensure the work is done safely and effectively."
        ]
    },
    {
        "tag": "time_for_yourself",
        "patterns": [
            "How to make time for yourself",
            "Importance of self-care",
            "Relaxation techniques"
        ],
        "responses": [
            "Making time for yourself is essential for mental well-being. Set aside moments for relaxation, hobbies, or simply doing nothing.",
            "Self-care involves nurturing your physical, emotional, and mental health to maintain balance and prevent burnout."
        ]
    },
    {
        "tag": "creative_writing",
        "patterns": [
            "Tell me about creative writing",
            "How to overcome writer's block",
            "Starting a blog"
        ],
        "responses": [
            "Creative writing includes storytelling, poetry, and fictional works. To overcome writer's block, take breaks, write regularly, and seek inspiration from other authors.",
            "Starting a blog allows you to share your thoughts, knowledge, and passions with a wider audience."
        ]
    },
    {
        "tag": "home_organization",
        "patterns": [
            "Tell me about home organization tips",
            "How to declutter your space",
            "Organizing small living spaces"
        ],
        "responses": [
            "Home organization tips include decluttering regularly, using storage solutions, and assigning specific places for items.",
            "For small living spaces, use multi-functional furniture and maximize vertical storage."
        ]
    },
    {
        "tag": "study_tips",
        "patterns": [
            "Tell me about effective study techniques",
            "How to stay focused while studying",
            "Preparing for exams"
        ],
        "responses": [
            "Effective study techniques include active learning, creating study schedules, and taking breaks to avoid burnout.",
            "To stay focused while studying, find a quiet and comfortable environment, avoid distractions, and set realistic goals."
        ]
    },
    {
        "tag": "positive_affirmations",
        "patterns": [
            "Tell me about positive affirmations",
            "How to practice self-affirmation",
            "Affirmations for confidence"
        ],
        "responses": [
            "Positive affirmations are powerful statements that promote self-belief and self-empowerment.",
            "Practicing self-affirmation involves repeating positive statements daily to reinforce a positive self-image."
        ]
    },
    {
        "tag": "photography",
        "patterns": [
            "Tell me about photography tips",
            "How to take better photos",
            "Different types of photography"
        ],
        "responses": [
            "Photography tips include understanding composition, lighting, and using manual settings on your camera.",
            "Different types of photography include portrait, landscape, wildlife, and street photography."
        ]
    },
    {
        "tag": "leadership_skills",
        "patterns": [
            "Tell me about developing leadership skills",
            "How to lead a team effectively",
            "Leadership qualities"
        ],
        "responses": [
            "Developing leadership skills involves continuous learning, setting a positive example, and empowering others.",
            "To lead a team effectively, communicate clearly, delegate tasks, and foster a collaborative and inclusive environment."
        ]
    },
    {
        "tag": "online_business",
        "patterns": [
            "Tell me about starting an online business",
            "How to build an e-commerce website",
            "Social media marketing"
        ],
        "responses": [
            "Starting an online business involves identifying a niche, creating a business plan, and choosing an e-commerce platform.",
            "Building an e-commerce website requires user-friendly design, secure payment options, and product descriptions."
        ]
    },
    {
        "tag": "positive_self-talk",
        "patterns": [
            "Tell me about positive self-talk",
            "How to practice self-compassion",
            "Building self-confidence"
        ],
        "responses": [
            "Positive self-talk involves replacing negative thoughts with supportive and affirming ones.",
            "Practicing self-compassion involves treating yourself with kindness and understanding during challenging times."
        ]
    },
    {
        "tag": "home_security",
        "patterns": [
            "Tell me about home security measures",
            "Installing a security system",
            "Creating a safe home environment"
        ],
        "responses": [
            "Home security measures include installing locks, security cameras, and motion sensor lights.",
            "A security system can provide peace of mind and deter potential intruders."
        ]
    },
    {
        "tag": "interpersonal_skills",
        "patterns": [
            "Tell me about interpersonal skills",
            "How to improve communication skills",
            "Respecting diverse perspectives"
        ],
        "responses": [
            "Interpersonal skills involve active listening, empathy, and adaptability in social interactions.",
            "Improving communication skills can enhance relationships and minimize misunderstandings."
        ]
    },
    {
        "tag": "work_life_balance",
        "patterns": [
            "Tell me about achieving work-life balance",
            "Setting boundaries between work and personal life",
            "The importance of self-care in maintaining balance"
        ],
        "responses": [
            "Achieving work-life balance involves prioritizing personal time, setting boundaries, and knowing when to take breaks.",
            "Self-care is essential for maintaining balance and preventing burnout."
        ]
    },
    {
        "tag": "team_building",
        "patterns": [
            "Tell me about team-building activities",
            "How to foster a positive team dynamic",
            "Improving communication within teams"
        ],
        "responses": [
            "Team-building activities can include team retreats, problem-solving exercises, and icebreaker games.",
            "Fostering a positive team dynamic requires open communication, trust, and recognizing individual strengths."
        ]
    },
    {
        "tag": "social_anxiety",
        "patterns": [
            "Tell me about coping with social anxiety",
            "Practicing relaxation techniques for social situations",
            "Seeking professional support for anxiety disorders"
        ],
        "responses": [
            "Coping with social anxiety involves challenging negative thoughts, gradual exposure to social situations, and learning relaxation techniques to manage anxiety symptoms.",
            "For severe social anxiety or anxiety disorders, seeking professional support, such as therapy or counseling, can be beneficial."
        ]
    },
    {
        "tag": "mind_control",
        "patterns": [
            "Exploring the concept of mind control",
            "Mind control in science fiction",
            "Psychic abilities and mental manipulation"
        ],
        "responses": [
            "Mind control is a sci-fi trope where individuals possess the power to control others' thoughts and actions.",
            "In reality, mind control remains a subject of science fiction and not a real-world capability."
        ]
    } 
]       

In [15]:
#Now let’s prepare the intents and train a Machine Learning model for the chatbot:

In [16]:
# Create the vectorizer and classifier
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

# Preprocess the data
tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

# training the model
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x, y)

LogisticRegression(max_iter=10000, random_state=0)

In [17]:
#Now let’s write a Python function to chat with the chatbot:

In [18]:
def chatbot(input_text):
    input_text = vectorizer.transform([input_text])
    tag = clf.predict(input_text)[0]
    for intent in intents:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
            return response

In [19]:
#Till now, we have created the chatbot. After running the code, you can interact with the chatbot in the terminal itself.

In [20]:
#So, here’s how we can deploy the chatbot using Python:

In [21]:
counter = 0

def main():
    global counter
    st.title("Chatbot")
    st.write("Welcome to the chatbot. Please type a message and press Enter to start the conversation.")

    counter += 1
    user_input = st.text_input("You:", key=f"user_input_{counter}")

    if user_input:
        response = chatbot(user_input)
        st.text_area("Chatbot:", value=response, height=100, max_chars=None, key=f"chatbot_response_{counter}")

        if response.lower() in ['goodbye', 'bye']:
            st.write("Thank you for chatting with me. Have a great day!")
            st.stop()

if __name__ == '__main__':
    main()

2024-12-06 23:07:34.619 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-06 23:07:34.621 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-06 23:07:34.623 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-06 23:07:34.623 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-06 23:07:34.623 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-06 23:07:34.623 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-06 23:07:34.628 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-06 23:07:34.628 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar